In [90]:
import pandas as pd
import re
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import spacy 
from IPython.display import display, HTML
from spacy import displacy
from transformers import pipeline


nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('pos_tag')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Error loading pos_tag: Package 'pos_tag' not found in
[nltk_data]     index
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[

True

In [28]:
df = pd.read_csv("C:\\Users\\admin\\OneDrive - BOMBAY COATED AND SPECIAL STEELS PRIVATE LIMITED\\Desktop\\customer_support_ticket_analyzer\\data\\customer_support_tickets_1.csv")
df.head()

,message_id,user_name,location,email_subject,message_body,created_at,true_category
0,M0001,Ryan Obrien,North Billy,Law from traditional now Mrs.,Reflect available century join outside. i cant...,2025-09-01 06:10:59,Feature Request
1,M0002,Jamie Salazar,South Kari,Require billion probably cut raise include now.,Try cause behind single project. Sport sound c...,2025-06-16 04:58:19,Other
2,M0003,Clinton Wallace,Port Paige,South maintain year firm.,While travel major strong pull. Us history lig...,2025-08-17 14:54:02,Feature Request
3,M0004,Christopher Olsen,Jeanville,Fill personal fire.,Can nothing force move free body stand approac...,2025-10-15 06:27:28,Praise
4,M0005,Alex Alexander,West Gail,Term authority offer feeling than.,Control skin fall. Left worker ready take prop...,2025-05-14 22:35:51,Praise


Load and inspect the dataset (CSV format). Identify	missing	values,	duplicates, and	column types.	

In [10]:
df.isnull().sum() # Check for missing values

message_id       0
user_name        0
location         0
email_subject    0
message_body     0
created_at       0
true_category    0
dtype: int64

In [15]:
df.duplicated() # Check for duplicate rows

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [16]:
df.dtypes # Check data types of each column

message_id       object
user_name        object
location         object
email_subject    object
message_body     object
created_at       object
true_category    object
dtype: object

Perform	deep text cleaning on 'message_body': remove emojis, HTML, repeated	punctuation, extra spaces,and fix common spelling errors.	

In [29]:
# Text Preprocessing
#remove the emojis
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['message_body'] = df['message_body'].apply(remove_emojis)
df['message_body'].head()

0    Reflect available century join outside. i cant...
1    Try cause behind single project. Sport sound c...
2    While travel major strong pull. Us history lig...
3    Can nothing force move free body stand approac...
4    Control skin fall. Left worker ready take prop...
Name: message_body, dtype: object

In [30]:
#remove the HTLM tags
def remove_html_tags(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

df['message_body'] = df['message_body'].apply(remove_html_tags)
df['message_body'].head()

0    Reflect available century join outside. i cant...
1    Try cause behind single project. Sport sound c...
2    While travel major strong pull. Us history lig...
3    Can nothing force move free body stand approac...
4    Control skin fall. Left worker ready take prop...
Name: message_body, dtype: object

In [32]:
#remove repeated punctuation and special characters 
def remove_special_characters(text):    
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

df['message_body'] = df['message_body'].apply(remove_special_characters)
df['message_body'].head()

0    Reflect available century join outside i cant ...
1    Try cause behind single project Sport sound cl...
2    While travel major strong pull Us history ligh...
3    Can nothing force move free body stand approac...
4    Control skin fall Left worker ready take prope...
Name: message_body, dtype: object

In [33]:
#remove space extra spaces
def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()
df['message_body'] = df['message_body'].apply(remove_extra_spaces)
df['message_body'].head()

0    Reflect available century join outside i cant ...
1    Try cause behind single project Sport sound cl...
2    While travel major strong pull Us history ligh...
3    Can nothing force move free body stand approac...
4    Control skin fall Left worker ready take prope...
Name: message_body, dtype: object

In [37]:
#fix spelling mistakes
def correct_spelling(text):
    return str(TextBlob(text).correct())
df['message_body'] = df['message_body'].apply(correct_spelling)
df['message_body'].head()


0    Reflect available century join outside i can l...
1    Cry cause behind single project Port sound cle...
2    While travel major strong pull Is history ligh...
3    An nothing force move free body stand approach...
4    Control skin fall Left worker ready take prope...
Name: message_body, dtype: object

Perform text preprocessing — apply tokenization, lowercasing, lemmatization, and POS tagging using spaCy or NLTK.

In [40]:
#apply tokenisation
def tokenize_text(text):
    return word_tokenize(text)
df['tokenized_message'] = df['message_body'].apply(tokenize_text)
df[['message_body', 'tokenized_message']].head()

,message_body,tokenized_message
0,Reflect available century join outside i can l...,"[Reflect, available, century, join, outside, i..."
1,Cry cause behind single project Port sound cle...,"[Cry, cause, behind, single, project, Port, so..."
2,While travel major strong pull Is history ligh...,"[While, travel, major, strong, pull, Is, histo..."
3,An nothing force move free body stand approach...,"[An, nothing, force, move, free, body, stand, ..."
4,Control skin fall Left worker ready take prope...,"[Control, skin, fall, Left, worker, ready, tak..."


In [41]:
#lowercasing the text
def lowercase_text(text):
    return text.lower()
df['message_body'] = df['message_body'].apply(lowercase_text)
df['message_body'].head()

0    reflect available century join outside i can l...
1    cry cause behind single project port sound cle...
2    while travel major strong pull is history ligh...
3    an nothing force move free body stand approach...
4    control skin fall left worker ready take prope...
Name: message_body, dtype: object

In [ ]:
#lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    tokens = word_tokenize(text)
    return [lemmatizer.lemmatize(token) for token in tokens]
df['lemmatized_message'] = df['message_body'].apply(lemmatize_text)
df[['message_body', 'lemmatized_message']].head()

In [46]:
#pos tagging
def pos_tagging(text): 
    tokens = word_tokenize(text)
    return pos_tag(tokens)
df['pos_tagged_message'] = df['message_body'].apply(pos_tagging)
df[['message_body', 'pos_tagged_message']].head()

,message_body,pos_tagged_message
0,reflect available century join outside i can l...,"[(reflect, NN), (available, JJ), (century, NN)..."
1,cry cause behind single project port sound cle...,"[(cry, NN), (cause, NN), (behind, IN), (single..."
2,while travel major strong pull is history ligh...,"[(while, IN), (travel, NN), (major, JJ), (stro..."
3,an nothing force move free body stand approach...,"[(an, DT), (nothing, NN), (force, NN), (move, ..."
4,control skin fall left worker ready take prope...,"[(control, NN), (skin, NN), (fall, NN), (left,..."


Extract key entities using Named Entity Recognition (NER) to identify product names, user names, locations, and issue types.

In [76]:
nlp = spacy.load("en_core_web_sm")

def extract_key_entities(text):
    if not isinstance(text, str) or not text.strip():
        return {
            "products": [],
            "user_names": [],
            "locations": [],
            "issue_types": []
        }

    doc = nlp(text)

    products = [ent.text for ent in doc.ents if ent.label_ == "PRODUCT"]
    user_names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    locations = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
    issue_types = [ent.text for ent in doc.ents if ent.label_ == "  NORP"]  

    return {
        "products": products,
        "user_names": user_names,
        "locations": locations,
        "issue_types": issue_types
    }
df['key_entities'] = df['message_body'].apply(extract_key_entities)
df[['message_body', 'key_entities']].head()

,message_body,key_entities
0,reflect available century join outside i can l...,"{'products': [], 'user_names': [], 'locations'..."
1,cry cause behind single project port sound cle...,"{'products': [], 'user_names': [], 'locations'..."
2,while travel major strong pull is history ligh...,"{'products': [], 'user_names': [], 'locations'..."
3,an nothing force move free body stand approach...,"{'products': [], 'user_names': [], 'locations'..."
4,control skin fall left worker ready take prope...,"{'products': [], 'user_names': [], 'locations'..."


Classify each message into categories such as Complaint, Bug Report, Feature Request, Praise, etc., using either traditional ML models or transformer-based models.

In [77]:
#classify each message into categories
def classify_message(text):
    text = text.lower()
    if "billing" in text or "invoice" in text or "payment" in text:
        return "Complaint"
    elif "technical" in text or "error" in text or "bug" in text:
        return "Bug Report"
    elif "account" in text or "login" in text or "password" in text:
        return "Feature Request"
    else:
        return "Praise"
    
df['message_category'] = df['message_body'].apply(classify_message)
df[['message_body', 'message_category']].head()

,message_body,message_category
0,reflect available century join outside i can l...,Feature Request
1,cry cause behind single project port sound cle...,Feature Request
2,while travel major strong pull is history ligh...,Praise
3,an nothing force move free body stand approach...,Bug Report
4,control skin fall left worker ready take prope...,Praise


In [95]:
from transformers import pipeline (classification, model = 'bert-large-cased-finetuned-conll03-english')

ner_pipeline = pipeline("ner", model='bert-large-cased-finetuned-conll03-english', grouped_entities=True)

def transformer_ner(text):
    return ner_pipeline(text)

df['transformer_ner'] = df['message_body'].apply(transformer_ner)
df[['message_body', 'transformer_ner']].head()

SyntaxError: invalid syntax (4239114422.py, line 1)

In [98]:
# ...existing code...
from transformers import torchpipeline

# use a valid model id on the Hub; e.g. dbmdz/bert-large-cased-finetuned-conll03-english
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)

def transformer_ner(text):
    if not isinstance(text, str) or not text:
        return []
    return ner_pipeline(text)

df['transformer_ner'] = df['message_body'].apply(transformer_ner)
df[['message_body', 'transformer_ner']].head()
# ...existing code...

ImportError: cannot import name 'torchpipeline' from 'transformers' (C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\transformers\__init__.py)

Summarize lengthy messages — for messages exceeding 100 words, apply text summarization using pretrained models like T5 or BART.

In [92]:
summarizer = pipeline("summarization",model="t5-base")
def summarize_text(text):
    return summarizer(text)

df['summary'] = df['message_body'].apply(summarize_text)
df[['message_body', 'summary']].head()

NameError: name 'torch' is not defined

Generate an output CSV file containing the cleaned text, predicted category, extracted entities, and summary.

In [ ]:
#generate csv file
df.to_csv("C:\\Users\\admin\\OneDrive - BOMBAY COATED AND SPECIAL STEELS PRIVATE LIMITED\\Desktop\\customer_support_ticket_analyzer\\data\\processed_customer_support_tickets.csv", index=False)
if os.path.exists("C:\\Users\\admin\\OneDrive - BOMBAY COATED AND SPECIAL STEELS PRIVATE LIMITED\\Desktop\\customer_support_ticket_analyzer\\data\\processed_customer_support_tickets.csv"):
    print("CSV file generated successfully.")

(Optional) Develop an interactive app using Streamlit or Gradio, with filters for category, city, or keyword-based search.